In [ ]:
#Create an Enterprise Geodatabse and import the XML pieces to test the model development in Khub DEV
#When re-iterating a deleted database, you can keep the existing sde connecitions

from arcpy import CreateEnterpriseGeodatabase_management, CreateDatabaseConnection_management, CreateDatabaseUser_management
from getpass import getpass

sapass = getpass('sysadmin password')
sdepass = getpass('sde password')
rhpass = getpass('RH password')
DBName = "KHub3"
CreateEnterpriseGeodatabase_management("SQL_Server", r"khdbtransdev\khub_dev", DBName, "DATABASE_AUTH", "geo_admin", sapass, "SDE_SCHEMA", "sde", sdepass, "", "C:/gisdata/keycodes/License10.6/sysgen/keycodes")
CreateDatabaseConnection_management(r"C:/gisdata/DBConnections/Khub/KHDev", "sde@"+DBName, "SQL_Server", r"khdbtransdev\khub_dev", "DATABASE_AUTH", "sde", sdepass, "SAVE_USERNAME", DBName)
CreateDatabaseConnection_management(r"C:/gisdata/DBConnections/Khub/KHDev", "sa@"+DBName, "SQL_Server", r"khdbtransdev\khub_dev", "DATABASE_AUTH", "geo_admin", sapass, "SAVE_USERNAME", DBName)
CreateDatabaseUser_management(r"C:/gisdata/DBConnections/Khub/KHDev/sa@"+DBName+".sde",  "DATABASE_USER", "rh", rhpass)
CreateDatabaseConnection_management(r"C:/gisdata/DBConnections/Khub/KHDev", "rh@"+DBName, "SQL_Server", r"khdbtransdev\khub_dev", "DATABASE_AUTH", "rh", rhpass, "SAVE_USERNAME", DBName)


In [ ]:
from arcpy import management, locref, conversion

sourceALRS = r'C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde'
proALRSdb = r'C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde'

projVert = "PROJCS['NAD_1983_Kansas_LCC_ftUS',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Lambert_Conformal_Conic'],PARAMETER['False_Easting',1312333.3333],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-98.25],PARAMETER['Standard_Parallel_1',37.5],PARAMETER['Standard_Parallel_2',39.5],PARAMETER['Latitude_Of_Origin',36.0],UNIT['Foot_US',0.3048006096012192]],VERTCS['NAVD88_height_(ftUS)',VDATUM['North_American_Vertical_Datum_1988'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Foot_US',0.3048006096012192]];-120729000 -94795000 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision"

management.CreateFeatureDataset(r"C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde", "ALRS", projVert)
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.Redline", proALRSdb+r"\KHub3.RH.ALRS", "Redline", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.Calibration_Point", proALRSdb+r"\KHub3.RH.ALRS", "Calibration_Point", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.Centerline", proALRSdb+r"\KHub3.RH.ALRS", "Centerline", '', '', "NETWORK_GEOMETRY")
conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH.LRS_County", proALRSdb+r"\KHub3.RH.ALRS", "LRS_County", '', '', "NETWORK_GEOMETRY")
conversion.TableToGeodatabase(r"C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde\KHUB.RH.Centerline_Sequence;C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde\KHUB.RH.Lrs_Edit_Log;C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde\KHUB.RH.LRS_EVENT_BEHAVIOR;C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde\KHUB.RH.Lrs_Locks;C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde\KHUB.RH.LRS_METADATA", r"C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde")

locref.ModifyLRSNetwork(proALRSdb+r"\KHub3.RH.ALRS\KHub3.RH.LRS_County", "RouteId", None, "LRSFromDate", "LRSToDate", "DO_NOT_DERIVE", '', "DO_NOT_INCLUDE", None, None, None, "MULTI_FIELD_ROUTE_ID", "County;Prefix;RouteNum;Suffix;InvDir;UniqueId")

to model existing events from feature classes, the feature classes must be in the ALRS Database and be M and Z aware


In [ ]:
from arcpy import env
env.Mtolerance = '0.00000062136995'
env.outputMFlag = 'Enabled'
env.OutputZFlag = 'Enabled'
env.XYTolerance = '0.001'
conversion.FeatureClassToFeatureClass(r"C:\gisdata\DBConnections\Khub\KHDev\rh@KHub2.sde\KHUB2.RH.ev_AccessControl", 
                                      r"C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde\KHub3.RH.ALRS", 
                                      "Access_Control3d", '', 
                                      '',
                                      "NETWORK_GEOMETRY")
# ERROR 130043: Feature class is not Z-aware.  Failed to execute (CreateLRSEventFromExistingDataset).
#have to change routeID text length to 13, matching routeID in LRS County
locref.CreateLRSEventFromExistingDataset(r"C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde\KHub3.RH.ALRS\KHub3.RH.LRS_County", r"C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde\KHub3.RH.ALRS\KHub3.RH.Access_Control3d", "EventID", "RouteID", "InventoryStartDate", "InventoryStartDate", "LocError", "FromMeasure", "ToMeasure", "NO_SPANS_ROUTES", None, "NO_STORE_ROUTE_NAME", None, None)

In [3]:
EventList = ['ev_AADT',
            'ev_AADT2012',
            'ev_AADT2013',
            'ev_AADT2014',
            'ev_AADT2015',
            'ev_AADT2016',
            'ev_AADTDirFactor',
            'ev_AADTExpansionFactor',
            'ev_AADTKFactor',
            'ev_AccessControl',
            'ev_AccessPermit',
            'ev_AccidentAnalysisDate',
            'ev_Accidents',  #crashes on accident layer
            'ev_AdministrativeOwner',
            'ev_AdvisorySpeed',
            'ev_AuxiliaryLane',
            'ev_CapacityHCM',
            'ev_CityJurisdiction',
            'ev_CollectorSection',
            'ev_CongressionalDistrict',
            'ev_County',
            'ev_CountyOnState',
            'ev_CrestVertSightDistNP',
            'ev_CrestVertSightDistP',
            'ev_Culverts',
            'ev_CurbType',
            'ev_DesignAccessControl',
            'ev_DesignStandShoulderWt',
            'ev_EPFS',
            'ev_EPFS2012',
            'ev_EPFS2013',
            'ev_EPFS2014',
            'ev_EPFS2015',
            'ev_EPFS2016',  #need to add additional EPFS Fields eventually?
            'ev_FieldInventory',
            'ev_FreightNetwork',
            'ev_FunctionalClass',
            'ev_GradientPercent',
            'ev_HorizontalCurve',
            'ev_HorizontalCurveFT',
            'ev_HPMSPavementCondition',
            'ev_HPMSReporting',
            'ev_HPMSSample',
            'ev_IndianNation',  #Check Metadata
            'ev_KDOTMaintenance',
            'ev_KDOTProjects',
            'ev_KDOTRouteClass',
            'ev_LegacyRoute', #need to check metadata
            'ev_MaintenanceResp',
            'ev_MajorGrading',
            'ev_ManagedLanes',
            'ev_Median',
            'ev_MedianBarrierType',
            'ev_MedianCrossoverType',
            'ev_MPOJurisdiction',
            'ev_MuniROW',
            'ev_NG911RouteStatus',
            'ev_NG911StreetName',
            'ev_NHSFederalAid',
            'ev_NonStateRouteStatus',
            'ev_NumberLanesAggregated',
            'ev_NumberThroughLanes',
            'ev_OnStreetParkingLeft',
            'ev_OnStreetParkingRight',
            'ev_OutdoorAdvertising',
            'ev_PassingLeft',
            'ev_PassingRight',
            'ev_PavementActivity',
            'ev_PavementConditionPSR',
            'ev_PavementCondSummary',
            'ev_PavementManagementSys',
            'ev_PriorityOptPvmtEquiv',
            'ev_ProjectTracking',
            'ev_RailroadCrossing',
            'ev_ReferencePost',
            'ev_RightOfWayLeft',
            'ev_RightOfWayMedian',
            'ev_RightOfWayRight',
            'ev_RockWedgeLeft',
            'ev_RockWedgeMedian',
            'ev_RockWedgeRight',
            'ev_RouteDesignation',
            'ev_RumbleStripCenterline',
            'ev_RumbleStripLeft',
            'ev_RumbleStripMedian',
            'ev_RumbleStripRight',
            'ev_SafetyAnalyst',
            'ev_ShoulderMaterialLeft',
            'ev_ShoulderMaterialMed',
            'ev_ShoulderMaterialRight',
            'ev_ShoulderWidthLeft',
            'ev_ShoulderWidthMedian',
            'ev_ShoulderWidthRight',
            'ev_SideSlopeLeft',
            'ev_SideSlopeMedian',
            'ev_SideSlopeRight',
            'ev_SNICE',
            'ev_SpeedLimit',
            'ev_STRAHNET',
            'ev_StripMapMajorGrading',
            'ev_Structure',
            'ev_StructureSegment',
            'ev_SupplementalGrading',
            'ev_SurfaceTypeNSRural',
            'ev_SurfaceWidthPaved',
            'ev_TerrainType',
            'ev_ThroughLaneWidth',
            'ev_TrafficSequence',
            'ev_TravelWayOperation',
            'ev_TruckRoute',
            'ev_TurnLeft',
            'ev_TurnRight',
            'ev_UrbanJurisdiction',
            'ev_WideningObstacle',
            'ev_WorkZone'
    ]


In [4]:

sourceALRS = r'C:\gisdata\DBConnections\Khub\Khub\rh@KHTransKHUB.sde'
proALRSdb = r'C:\gisdata\DBConnections\Khub\KHDev\rh@KHub3.sde'
from arcpy import env, locref, conversion, Exists, management

env.Mtolerance = '0.00000062136995'
env.outputMFlag = 'Enabled'
env.OutputZFlag = 'Enabled'
env.XYTolerance = '0.001'

for event in EventList:
    print(event)
    importfeature = event
    if Exists(proALRSdb+r"\KHub3.RH.ALRS\KHub3.RH."+event):
        print(importfeature + " Exists")
    else:
        print('converting....')
        conversion.FeatureClassToFeatureClass(sourceALRS+r"\KHUB.RH."+event, proALRSdb+r"\KHub3.RH.ALRS", event, '1=0', '',"NETWORK_GEOMETRY")
        print ('altering...')
        management.AlterField(proALRSdb+r'\KHub3.RH.ALRS\\'+ event, "RouteID", "RouteID", "RouteID", "TEXT", "13", "NULLABLE")
        management.Append(sourceALRS+r"\KHUB.RH."+event, proALRSdb+r'\KHub3.RH.ALRS\\'+ event)

        print('creating event')
        locref.CreateLRSEventFromExistingDataset(proALRSdb+r"\KHub3.RH.ALRS\KHub3.RH.LRS_County", proALRSdb+r"\KHub3.RH.ALRS\KHub3.RH."+event, "EventID", "RouteID", "LRSFromDate", "LRSToDate", "LocError", "FromMeasure", "ToMeasure", "NO_SPANS_ROUTES", None, "NO_STORE_ROUTE_NAME", None, None)
        
                              
                              

The version of the binaries in the active Python environment do
  not match the version of the binaries of the ArcGIS Pro application.

Active environment path: C:\Users\kyleg\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone1

To create a new environment that is up to date with arcgispro-py3:
 1. Generate a list of additional packages installed in your current environment,
    with the conda command:
        > conda env export > env.yaml
 2. (Optional) If you have additional dependencies installed through pip,
    find those with pip freeze:
        > pip freeze > requirements.txt
 2. Create a new environment by cloning arcgispro-py3, and activate it:
        > conda create --clone arcgispro-py3 --name my-env --pinned
        > activate my-env
 3. Add back missing conda packages from your current environment:
        > conda env update -n my-env -f env.yaml
 4. (Optional) Add back missing packages from pip:
        > pip install -r requirements.txt
 5. (Optional) Make it the default f

ImportError: version mis-match

In [ ]:
I